# Web Scraper for r/wallstreetbets

I largely follow the same workflow as in Assignment 2.

In [1]:
import boto3
import requests
import datetime
import mysql.connector
import json
import csv
import os
import pandas as pd
import fastparquet

## Create RDS

In [2]:
# Create RDS on AWS
rds = boto3.client("rds", region_name="us-east-1")

try:
    response = rds.create_db_instance(DBInstanceIdentifier='relational-db',
                                      DBName='wsb_rds',
                                      MasterUsername='username',
                                      MasterUserPassword='password',
                                      DBInstanceClass='db.t2.micro',
                                      Engine='MySQL',
                                      AllocatedStorage=5)
except:
    print("DB already exists")

# Wait until db is ready to continue
rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier="relational-db")

# Get relevant information about db
db = rds.describe_db_instances()['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

# Check DB is available
print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT)  

DB already exists
relational-db is available at relational-db.cpwt7pwqqywi.us-east-1.rds.amazonaws.com on Port 3306


In [3]:
# edit security settings
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust Permissions for that security group so that we can access it on Port 3306
# If already SG is already adjusted, print this out
try:
    ec2 = boto3.client('ec2', region_name="us-east-1")
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Permissions already adjusted.


In [72]:
# launch connection
conn =  mysql.connector.connect(host=ENDPOINT, user="username", passwd="password", port=PORT, database="wsb_rds")
cur = conn.cursor()

## Create Table to store Scraped Data

In [5]:
# create table
create_wsb_table = """ 
                        CREATE TABLE IF NOT EXISTS wsb_posts (
                            link TEXT,
                            title TEXT,
                            text TEXT,
                            author TEXT,
                            category TEXT,
                            n_comments INT,
                            upvotes INT,
                            upvote_ratio INT,
                            n_awards INT,
                            date INT,
                            PRIMARY KEY (date)
                            )
                        """
for op in [create_wsb_table]:
    cur.execute(op)

## Create Lambda Function

In [61]:
# create lambda function
# Create Lambda client
aws_lambda = boto3.client('lambda', region_name="us-east-1")

# Access our class IAM role, which allows Lambda
# to interact with other AWS resources
iam_client = boto3.client('iam', region_name="us-east-1")
role = iam_client.get_role(RoleName='LabRole')

# Read pyfile
with open("wsb_lambda.zip", "rb") as f:
    lambda_file = f.read()

# Create lambda function
try:
    response = aws_lambda.create_function(FunctionName="wsb_lambda",
                                          Runtime='python3.9',
                                          Role=role['Role']['Arn'],
                                          Handler='wsb_lambda.lambda_handler',
                                          Code=dict(ZipFile=lambda_file),
                                          Timeout=300)
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
    FunctionName="wsb_lambda",
    ZipFile=lambda_file)
    print("Lambda successfully updated")

Lambda successfully updated


## Generate List of URLs to Scrape

In [8]:
# List of urls to scrape
start = datetime.datetime.strptime("24-05-2021", "%d-%m-%Y")
end = datetime.datetime.strptime("30-05-2022", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
url_list = []
for idx, date in enumerate(date_generated):
    if idx < len(date_generated) - 1:
        before_date = date_generated[idx + 1].strftime("%Y-%m-%d")
        after_date = date_generated[idx].strftime("%Y-%m-%d")
        url = f"https://api.pushshift.io/reddit/submission/search/?subreddit=wallstreetbets&after={after_date}&before={before_date}&size=500&sort_type=score"
        url_list.append(url)

## Split into Batches

In [68]:
# split urls into batches
n = len(url_list) // 10 # subdivide list of ISBNs into 10 equal batches
url_batches = [{'url_list': url_list[i:i + n]} for i in range(0, len(url_list), n)]

In [69]:
len(url_batches)

10

## Create Step Function

In [29]:
# setup step function
!python "wsb_sfn.py"

In [30]:
# View state machine
sfn = boto3.client('stepfunctions', region_name="us-east-1")
response = sfn.list_state_machines()
print(response)

{'stateMachines': [{'stateMachineArn': 'arn:aws:states:us-east-1:227666994724:stateMachine:wsb_sfn', 'name': 'wsb_sfn', 'type': 'EXPRESS', 'creationDate': datetime.datetime(2022, 5, 31, 9, 21, 16, 82000, tzinfo=tzlocal())}], 'ResponseMetadata': {'RequestId': 'ea1cb01c-b5f4-44f5-b989-f346a6aa3884', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ea1cb01c-b5f4-44f5-b989-f346a6aa3884', 'date': 'Tue, 31 May 2022 14:21:18 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '166'}, 'RetryAttempts': 0}}


## Split Batches Across Lambda Workers with Step Function

In [70]:
# run scraper

# Get arn for Step Function state machine
response = sfn.list_state_machines()
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'wsb_sfn'][0]

# Spread URL batches across Lambda workers
response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name="wsb_sfn",
    input=json.dumps(url_batches))

## View Number of Posts Scraped

In [73]:
cur.execute('''SELECT * FROM wsb_posts''')
query_results = cur.fetchall()
len(query_results)

10537

## Write Results to CSV and Upload to S3

In [107]:
fp = open('wsb_batch_2.csv', 'w', encoding="utf-8")
myFile = csv.writer(fp)
myFile.writerows(query_results)
fp.close()

In [108]:
# read contents of csv file
file = pd.read_csv("wsb_batch_2.csv")
  
# adding header
headerList = ["link", "title", "text", "author", "category", "n_comments", "upvotes", "upvote_ratio", "n_awards", "date"]
  
# converting data frame to csv
file.to_csv("wsb_batch_2.csv", header=headerList, index=False)

# convert data frame to parquet
file.to_parquet("wsb_batch_2.parquet",
                engine="fastparquet")

## Create S3 Bucket and Place CSV

In [110]:
# create S3 Bucket
s3 = boto3.resource("s3", region_name="us-east-1")
# Must use unique name
# s3.create_bucket(Bucket="thiyaghessan-wsb")

In [88]:
# put object
bucket = 'thiyaghessan-wsb'
filename = 'wsb_batch_2.csv'
s3.meta.client.upload_file(Filename = filename, Bucket= bucket, Key = filename)

In [111]:
# upload parquet form
bucket = 'thiyaghessan-wsb'
filename = 'wsb_batch_2.parquet'
s3.meta.client.upload_file(Filename = filename, Bucket= bucket, Key = filename)

## Delete All Requested Resources

In [35]:
# delete resources
# delete database
response = rds.delete_db_instance(DBInstanceIdentifier='relational-db',
                       SkipFinalSnapshot=True
                      )
print(response['DBInstance']['DBInstanceStatus'])

# wait until DB is deleted before proceeding
rds.get_waiter('db_instance_deleted').wait(DBInstanceIdentifier='relational-db')
print("RDS Database has been deleted")

# Delete Lambda Function
try:
    aws_lambda.delete_function(FunctionName="wsb_lambda")
except:
    print("Lambda function already deleted")
    
# Delete Step Function
try:
    sfn.delete_state_machine(stateMachineArn="arn:aws:states:us-east-1:227666994724:stateMachine:wsb_sfn")
except:
    print("Step Function Already deleted")

deleting
RDS Database has been deleted
